# 02b — Ingest JSONL to Cosmos DB (Gremlin)
Cosmos DB Gremlin does not support direct file uploads to a graph container. This loader reads JSON Lines files and upserts vertices/edges via Gremlin API.

In [ ]:

import json, time, glob
from gremlin_python.driver import client, serializer
from gremlin_python.process.traversal import __

with open('../config/cosmos_gremlin_config.json') as f:
    cfg = json.load(f)

gclient = client.Client(
    cfg['endpoint'], 'g',
    username=f"/dbs/{cfg['database']}/colls/{cfg['graph']}",
    password=cfg['primary_key'],
    message_serializer=serializer.GraphSONSerializersV2d0()
)

def run(q, max_retries=5):
    for i in range(max_retries):
        try:
            return gclient.submitAsync(q).result().all().result()
        except Exception as e:
            if i==max_retries-1: raise
            time.sleep(0.5*(2**i))

def upsert_vertex(v):
    label=v['label']; vid=v['id']; pk=v.get('pk','VERT'); props=v.get('properties',{})
    q=f"g.V('{vid}').fold().coalesce(unfold(), addV('{label}').property(id,'{vid}').property('pk','{pk}'))"
    for k, val in props.items():
        sval=str(val).replace("'","\\'")
        q+=f".property('{k}','{sval}')"
    run(q)

def upsert_edge(e):
    out_id=e['outV']; in_id=e['inV']; label=e['label']; pk=e.get('pk','EDGE')
    q=(f"g.V('{out_id}').as('a').V('{in_id}').as('b')"
       f".coalesce(__.E().hasLabel('{label}').where(outV().hasId('{out_id}')).where(inV().hasId('{in_id}')),"
       f"addE('{label}').from('a').to('b').property('pk','{pk}'))")
    run(q)

# Load vertices first
for fp in sorted(glob.glob('../datasets_json/vertices_*.jsonl')):
    print('Loading', fp)
    with open(fp, 'r') as f:
        for line in f:
            v=json.loads(line)
            upsert_vertex(v)

# Then edges
for fp in sorted(glob.glob('../datasets_json/edges_*.jsonl')):
    print('Loading', fp)
    with open(fp, 'r') as f:
        for line in f:
            e=json.loads(line)
            upsert_edge(e)

gclient.close(); print('JSONL ingestion complete.')
